In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep
import os
import psutil
import inspect

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks
kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


with server

In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
                           reset=True, 
                           n_consumers=3,
                           create_producer=True,
                           queue_port=3322)
                           
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Staging Protocol Changes...
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

setup: compute train-test overlap between a very large train set and a smaller test set. Small test is still to big for memory, so we split it into 54 parts. We keep 1 of those parts in memory. We dont keep the train set in memory, but read and compare with 1/54 parts line by line. Each part takes ~30 hours, but we can run 54 processes in parallel.

# Setup syft functions

## Dataset

In [3]:
x = ActionObject.from_obj([1,2])
x_ptr = x.send(client)

## Batch function

In [4]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    print("done")
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

SyftSuccess: User Code Submitted

## Main function

In [6]:
@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
#     results = [x.wait().get() for x in jobs]
    return 1
#     return sum(results)

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

# Approve & run

In [7]:
client.code.request_code_execution(process_all)
client.requests[-1].approve()

Request approved for domain test-domain-helm2


SyftSuccess: Request 688d6039acd84974aee10fc466858a91 changes applied

Request approved for domain test-domain-helm2


SyftSuccess: Request 688d6039acd84974aee10fc466858a91 changes applied

In [8]:
job = client.code.process_all(x=x_ptr, blocking=False)
sleep(5)

FUNCTION LOG (bf51f951b7c24d1f98de299513dc06c4): Launching jobs
FUNCTION LOG (bf51f951b7c24d1f98de299513dc06c4): Launching jobs


LAUNCHING JOB process_batch
LAUNCHING JOB process_batch
LAUNCHING JOB process_batch
LAUNCHING JOB process_batch


FUNCTION LOG (bf51f951b7c24d1f98de299513dc06c4): starting aggregation
FUNCTION LOG (bf51f951b7c24d1f98de299513dc06c4): Done
FUNCTION LOG (bf51f951b7c24d1f98de299513dc06c4): starting aggregation
FUNCTION LOG (bf51f951b7c24d1f98de299513dc06c4): Done
FUNCTION LOG (3320c11138e646af8e01639195804201): starting batch 2
FUNCTION LOG (a6f649bc8abd4e7cbe3061c0c2a43879): starting batch 1
FUNCTION LOG (3320c11138e646af8e01639195804201): starting batch 2
FUNCTION LOG (a6f649bc8abd4e7cbe3061c0c2a43879): starting batch 1
FUNCTION LOG (3320c11138e646af8e01639195804201): done
FUNCTION LOG (a6f649bc8abd4e7cbe3061c0c2a43879): done
FUNCTION LOG (3320c11138e646af8e01639195804201): done
FUNCTION LOG (a6f649bc8abd4e7cbe3061c0c2a43879): done


In [9]:
job.subjobs

[syft.service.job.job_stash.Job, syft.service.job.job_stash.Job]

[syft.service.job.job_stash.Job, syft.service.job.job_stash.Job]

In [10]:
res = job.wait().get()

In [11]:
res

1

1

In [12]:
job.subjobs[0].logs()

starting batch 1
done

starting batch 1
done



In [13]:
job.subjobs[1].logs()

starting batch 2
done

starting batch 2
done

